<a href="https://colab.research.google.com/github/kootr/ml-study-session/blob/main/bq_recommendation_mf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/kootr/ml-study-session/blob/main/20220730_reccomendation_mf/bq_recommendation_mf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [5]:
from google.cloud import bigquery
import pandas as pd

In [6]:
client = bigquery.Client(location="US", project="ml-session")

In [7]:
# GCP認証　(VertexAI workbench：必要なし、Googole collabo:必要、ローカル: 必要（Jsonクレデンシャルファイルを配置する）　)
import os
import sys

IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks（おそらくVertexAI Workbenchのこと）, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # どれも当てはまらない場合はローカル実行として、クレデンシャルファイルを指定
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '/hoge/fuga/key_file.json'

In [8]:
# テストクエリ
query = """
SELECT
  vendor_id,
  passenger_count,
  trip_distance,
  rate_code,
  payment_type,
  total_amount,
  tip_amount
FROM
  `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE tip_amount >= 0
LIMIT 100
"""
query_job = client.query(
    query,
     location="US",
)

df = query_job.to_dataframe()
df.head(5)

,vendor_id,passenger_count,trip_distance,rate_code,payment_type,total_amount,tip_amount
0,2,1,37.06,1,1,181.8,35.75
1,1,1,7.5,1,1,33.95,5.65
2,1,1,13.7,1,1,54.8,9.5
3,2,1,10.15,1,1,45.38,9.08
4,2,1,11.32,1,1,44.8,4


### 参考にしたTutorial 
https://cloud.google.com/bigquery-ml/docs/bigqueryml-mf-explicit-tutorial#find_all_the_item_ratings_for_a_set_of_users

In [10]:
# まずはデータセットを作成

dataset_id = "session13"
dataset = bigquery.Dataset(f'ml-session.{dataset_id}')
dataset.location = "US"

dataset = client.create_dataset(dataset)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

In [ ]:
# チュートリアル通り、movielensのレーティングデータ、タイトルデータをダウンロード、区切り文字加工
!curl -O 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
!unzip ml-1m.zip

!sed 's/::/,/g' ml-1m/ratings.dat > ratings.csv
!sed 's/::/@/g' ml-1m/movies.dat > movie_titles.csv # 区切り文字 @ とする

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5778k  100 5778k    0     0  11.5M      0 --:--:-- --:--:-- --:--:-- 11.5M
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
!head ratings.csv

1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
1,1197,3,978302268
1,1287,5,978302039
1,2804,5,978300719
1,594,4,978302268
1,919,4,978301368


In [ ]:
!head movie_titles.csv

1@Toy Story (1995)@Animation|Children's|Comedy
2@Jumanji (1995)@Adventure|Children's|Fantasy
3@Grumpier Old Men (1995)@Comedy|Romance
4@Waiting to Exhale (1995)@Comedy|Drama
5@Father of the Bride Part II (1995)@Comedy
6@Heat (1995)@Action|Crime|Thriller
7@Sabrina (1995)@Comedy|Romance
8@Tom and Huck (1995)@Adventure|Children's
9@Sudden Death (1995)@Action
10@GoldenEye (1995)@Action|Adventure|Thriller


In [ ]:
!cat movie_titles.csv | wc -l

3883


In [ ]:
# BigQuery clientを使ってテーブル作成、CSVインポート
def load_local_csv_to_bigquery(csv_file_path, table_id, schema, field_delimiter=','):
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        schema=schema,
        field_delimiter=field_delimiter
    )

    with open(csv_file_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)
    job.result()

In [11]:
rating_table ="ratings"
title_table ="movie_titles"

In [ ]:
# Create and load table for ratings.csv

table_id = f"{dataset_id}.{rating_table}"
csv_file_path = "./ratings.csv"
schema=[
        bigquery.SchemaField("user_id", "INT64"),
        bigquery.SchemaField("item_id", "INT64"),
        bigquery.SchemaField("rating", "FLOAT64"),
        bigquery.SchemaField("timestamp", "TIMESTAMP"),
    ]
load_local_csv_to_bigquery(csv_file_path, table_id, schema)

In [ ]:
# Create and load table for movie_titles.csv

table_id = f"{dataset_id}.{title_table}"
csv_file_path = "./movie_titles.csv"
schema=[
        bigquery.SchemaField("movie_id", "INT64"),
        bigquery.SchemaField("movie_title", "STRING"),
        bigquery.SchemaField("genre", "STRING"),
    ]
# 区切り文字 @ とする
load_local_csv_to_bigquery(csv_file_path, table_id, schema,'@')

In [12]:
# （本筋とは別）レーティングの基本情報を調べる

# rating_table
query = f"""
#standardSQL
SELECT
  user_id, item_id, rating
FROM
  {dataset_id}.{rating_table}
"""
query_job = client.query(
    query,
    location="US",
)
df_rating = query_job.to_dataframe()
df_rating.head()

# タイトルごとのレーティング取得
query = f"""
#standardSQL
SELECT
  {rating_table}.item_id,
  {title_table}.movie_title,
  {rating_table}.rating,
FROM
  {dataset_id}.{rating_table}
  LEFT JOIN {dataset_id}.{title_table} ON {rating_table}.item_id = {title_table}.movie_id
"""
query_job = client.query(
    query,
    location="US",
)
df_movie_ratings = query_job.to_dataframe()
df_movie_ratings.head()
test = df_movie_ratings.groupby('movie_title').agg(['count', 'mean'])
#うまくマルチインデックスの集計ができなかった
df_movie_ratings.groupby('movie_title').agg(['count'])

,item_id,rating
,count,count
movie_title,,
"$1,000,000 Duck (1971)",37,37
'Night Mother (1986),70,70
'Til There Was You (1997),52,52
"'burbs, The (1989)",303,303
...And Justice for All (1979),199,199
...,...,...
"Zed & Two Noughts, A (1985)",29,29
Zero Effect (1998),301,301


In [ ]:
df_movie_ratings

,item_id,movie_title,rating
0,1213,GoodFellas (1990),2.0
1,434,Cliffhanger (1993),2.0
2,3107,Backdraft (1991),2.0
3,902,Breakfast at Tiffany's (1961),2.0
4,3256,Patriot Games (1992),2.0
...,...,...,...
1000204,3388,Harry and the Hendersons (1987),1.0
1000205,2751,From the Hip (1987),1.0
1000206,2794,European Vacation (1985),1.0
1000207,2003,Gremlins (1984),1.0


In [ ]:
# 以下のクエリを実行するにはスロットの購入が必要。
# https://cloud.google.com/bigquery/docs/reservations-intro?hl=ja
# スロット購入の際はFlexプランの選択が妥当。実行後、スロットを削除する。
# BQスロットコミットメント購入→予約作成→予約割り当て→クエリ実行→予約割り当て削除→予約削除→スロットコミットメント削除

In [ ]:
# モデル作成　13分かかります。　途中経過を表示できるように修正したい。
model_name = "my_explicit_mf_model"
query = f"""
#standardSQL
CREATE OR REPLACE MODEL {dataset_id}.{model_name}
OPTIONS
  (model_type='matrix_factorization',
   user_col='user_id',
   item_col='item_id',
   l2_reg=9.83,
   num_factors=34,
   model_registry='vertex_ai',
   vertex_ai_model_id='movie_recommend',
   vertex_ai_model_version_aliases=['experimental']) AS
SELECT
  user_id,
  item_id,
  rating
FROM {dataset_id}.{rating_table}
"""

query_job = client.query(
    query,
    location="US",
)

In [ ]:
# Evaluate the model
query = f"""
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL {dataset_id}.{model_name},
    (
    SELECT
      user_id,
      item_id,
      rating
     FROM
      {dataset_id}.{rating_table})
      )
"""

query_job = client.query(
    query,
    location="US",
)

df = query_job.to_dataframe()

In [ ]:
df = query_job.to_dataframe()
df

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.48596,0.396012,0.025547,0.390904,0.682661,0.682661


In [ ]:
# 5ユーザーに対して、全ての映画のレーティングを予測する
query = f"""
#standardSQL
SELECT
  *
FROM
  ML.RECOMMEND(MODEL {dataset_id}.{model_name},
    (
    SELECT
      DISTINCT(user_id)
    FROM
      {dataset_id}.{rating_table}
    LIMIT 5))
"""
query_job = client.query(
    query,
    location="US",
)
df = query_job.to_dataframe()
df

,predicted_rating,user_id,item_id
0,1.804184,2,2561
1,3.902917,2,3329
2,2.791661,2,258
3,2.755314,2,3842
4,3.199996,2,1284
...,...,...,...
18525,4.548385,5,246
18526,3.774170,5,1785
18527,3.981500,5,506
18528,2.993428,5,3322


In [ ]:
# 全ユーザーに対して、全ての映画のレーティングを予測しテーブルに保存する（全ユーザー数×全映画数なので行数が膨大になります）
prediction_result_table = "all_prediction_result"
query = f"""
#standardSQL
CREATE OR REPLACE TABLE {dataset_id}.{prediction_result_table}
OPTIONS() AS
SELECT
  *
FROM
  ML.RECOMMEND(MODEL {dataset_id}.{model_name})
"""
query_job = client.query(
    query,
    location="US",
)

In [ ]:
# ユーザーごとに予測レートが高い作品上位５つを提示
query = f"""
#standardSQL
SELECT
  user_id,
  ARRAY_AGG(STRUCT(movie_title, genre, predicted_rating)
ORDER BY predicted_rating DESC LIMIT 5)
FROM (
SELECT
  user_id,
  item_id,
  predicted_rating,
  movie_title,
  genre
FROM
  {dataset_id}.{prediction_result_table}
JOIN
  {dataset_id}.{title_table}
ON
  item_id = movie_id)
GROUP BY
  user_id
"""
query_job = client.query(
    query,
    location="US",
)
df_movie_recommend = query_job.to_dataframe()
df_movie_recommend.head()

,user_id,f0_
0,1386,"[{'movie_title': 'Big Carnival, The (1951)', '..."
1,1771,"[{'movie_title': 'Song of Freedom (1936)', 'ge..."
2,5096,"[{'movie_title': 'Cup, The (Phörpa) (1999)', '..."
3,1449,"[{'movie_title': 'Song of Freedom (1936)', 'ge..."
4,453,"[{'movie_title': 'Two Family House (2000)', 'g..."


In [ ]:
# 発展：自分のレーティング情報から予測を作成
# 同じフォルダに配置されている30件の評価データを読み込みテーブル作成 my_ratings.csv

# Create and load table for my_ratings.csv
my_rating_table = "my_ratings"
table_id = f"{dataset_id}.{my_rating_table}"
csv_file_path = "./my_ratings.csv"
schema=[
        bigquery.SchemaField("user_id", "INT64"),
        bigquery.SchemaField("item_id", "INT64"),
        bigquery.SchemaField("rating", "FLOAT64"),
        # bigquery.SchemaField("timestamp", "TIMESTAMP"),
    ]
load_local_csv_to_bigquery(csv_file_path, table_id, schema)


query = f"""
#standardSQL
SELECT
  *
FROM
  ML.RECOMMEND(MODEL {dataset_id}.{model_name},
    (
SELECT
      COALESCE({my_rating_table}.user_id, 9999) user_id,
      {my_rating_table}.rating,
      {title_table}.movie_id item_id,
      {title_table}.movie_title,
      {title_table}.genre,
    FROM
      {dataset_id}.{title_table} LEFT JOIN {dataset_id}.{my_rating_table} ON {title_table}.movie_id = {my_rating_table}.item_id
    
    ))
    ORDER BY predicted_rating DESC
"""

query_job = client.query(
    query,
    location="US",
)
df_movie_recommend = query_job.to_dataframe()
df_movie_recommend.head()

In [ ]:
# ジャンルがレーティングに与える影響の評価
query = f"""
#standardSQL
SELECT
  factor,
  ARRAY_AGG(STRUCT(feature, genre,
      weight)
  ORDER BY
    weight DESC
  LIMIT
    10) AS weights
FROM (
  SELECT
    * EXCEPT(factor_weights)
  FROM (
    SELECT
      *
    FROM (
      SELECT
        factor_weights,
        CAST(feature AS INT64) as feature
      FROM
        ML.WEIGHTS(model {dataset_id}.{model_name})
      WHERE
        processed_input= 'item_id')
    JOIN
      {dataset_id}.{title_table}
    ON
      feature = movie_id) weights
  CROSS JOIN
    UNNEST(weights.factor_weights)
  ORDER BY
    feature,
    weight DESC)
GROUP BY
  factor
"""
query_job = client.query(
    query,
    location="US",
)
df_genre_factor = query_job.to_dataframe()
df_genre_factor.head()

,factor,weights
0,34,"[{'feature': 806, 'genre': 'Drama', 'weight': ..."
1,33,"[{'feature': 2893, 'genre': 'Action|Drama', 'w..."
2,32,"[{'feature': 3544, 'genre': 'Comedy', 'weight'..."
3,31,"[{'feature': 2711, 'genre': 'Drama', 'weight':..."
4,30,"[{'feature': 3577, 'genre': 'Drama', 'weight':..."


In [12]:
# リソース削除 TBA